In [1]:
from cosecha_colectiva import miscelaneous as m
m.try_find_conf_file()

from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import config
from cosecha_colectiva import CAF_DB_Monitor as CDM
from cosecha_colectiva import test_cosecha_api as tca

import logging
import pathlib
from datetime import datetime
import pandas as pd

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2023-05-21 19:32:21,922][INFO]: Connecting b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306
[2023-05-21 19:32:22,972][INFO]: Connected b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306


In [2]:
#xls_name = 'CAF_GrupoTesteador.xlsx'
#xls_name = 'CAF_GrupoAstronomo.xlsx'
xls_name = 'CAF_GrupoDomador.xlsx'
xls_name = 'CAF_GrupoCafam.xlsx'
xls_name = 'GrupoCafam.xlsx'
#xls_name = 'CAF_GrupoHidalgo.xlsx'

In [3]:
id_grupo = qc.get_grupo_id_by_name(xls_name.split('.')[0])
print(id_grupo)

1744


In [10]:
qc.delete_grupo(id_grupo, solo_sesiones=True, force_delete=True)

([{'Transaccion_prestamo_id': 5194}, {'Transaccion_prestamo_id': 5204}, {'Transaccion_prestamo_id': 5214}, {'Transaccion_prestamo_id': 5224}, {'Transaccion_prestamo_id': 5234}, {'Transaccion_prestamo_id': 5244}, {'Transaccion_prestamo_id': 5254}], 'transaccion_prestamos')
DELETE from heroku_59f792929604e63.transaccion_prestamos WHERE Transaccion_prestamo_id in (5194, 5204, 5214, 5224, 5234, 5244, 5254)
([{'Transaccion_id': 22564}, {'Transaccion_id': 22574}, {'Transaccion_id': 22584}, {'Transaccion_id': 22594}, {'Transaccion_id': 22604}, {'Transaccion_id': 22614}, {'Transaccion_id': 22624}, {'Transaccion_id': 22634}, {'Transaccion_id': 22644}, {'Transaccion_id': 22654}, {'Transaccion_id': 22664}, {'Transaccion_id': 22674}, {'Transaccion_id': 22684}, {'Transaccion_id': 22694}, {'Transaccion_id': 22704}], 'transacciones')
DELETE from heroku_59f792929604e63.transacciones WHERE Transaccion_id in (22564, 22574, 22584, 22594, 22604, 22614, 22624, 22634, 22644, 22654, 22664, 22674, 22684, 2269

In [11]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'_Sesiones.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [12]:
tca.CAF_API_sessions_tester.poner_corriente_grupo(xls_name)

{'COMPRA_ACCIONES':      COMPRA_ACCIONES
1704          [2200]
1834          [1511]
1844          [4750]
1854          [1423]
1864          [1459]
1874           [454]
1884           [287]
1894          [2200]
1904          [1600]
1914          [1300]
1924            [30], 'PAGO_MULTA': Empty DataFrame
Columns: [PAGO_MULTA]
Index: [], 'ABONO':           ABONO
1704  [11025.0]
1834  [22179.0]
1844  [22000.0]
1854   [3228.0]
1864  [17837.5]
1894  [32624.0]
1914  [1734.75], 'PRÉSTAMO':         PRÉSTAMO AMPLIACIÓN NUM_SESIONES
1704   [25000.0]        [0]          [9]
1834   [28000.0]        [0]          [8]
1844   [30000.0]        [0]         [20]
1854   [51500.0]        [0]         [47]
1864   [45000.0]        [0]         [24]
1894   [70000.0]        [0]          [4]
1914   [20000.0]        [0]         [13]
1924  [19938.67]        [0]        [102], 'AMPLIACIÓN': Empty DataFrame
Columns: [AMPLIACIÓN]
Index: [], 'NUM_SESIONES':      NUM_SESIONES
1704          [9]
1834          [8]
1844       

In [4]:


id_grupo, admin_header = tca.CAF_API_general.get_grupo_id_admin_header_xls(xls_name)
sesion_list = qc.get_all_sesiones_grupo(id_grupo)

dict_users = qc.get_socios_grupo(id_grupo)
dict_session = m.read_transform_info_xls(xls_name, 4, dict_users, type_xls="NEW", hoja="EstadisticaInicial")

print(dict_session)

{'COMPRA_ACCIONES': Empty DataFrame
Columns: [COMPRA_ACCIONES]
Index: [], 'PAGO_MULTA': Empty DataFrame
Columns: [PAGO_MULTA]
Index: [], 'ABONO': Empty DataFrame
Columns: [ABONO]
Index: [], 'PRÉSTAMO':      PRÉSTAMO AMPLIACIÓN NUM_SESIONES
1704    [1.0]        [0]          [6]
1834    [1.0]        [0]          [6]
1844    [1.0]        [0]          [6]
1854    [2.0]        [0]          [6]
1864    [2.0]        [0]          [6]
1894    [1.0]        [0]          [6]
1914    [3.0]        [0]          [6]
1924    [4.0]        [0]          [6], 'AMPLIACIÓN': Empty DataFrame
Columns: [AMPLIACIÓN]
Index: [], 'NUM_SESIONES': Empty DataFrame
Columns: [NUM_SESIONES]
Index: [], 'RETIRO_ACCIONES': Empty DataFrame
Columns: [RETIRO_ACCIONES]
Index: [], 'MULTAS': Empty DataFrame
Columns: [MULTAS]
Index: [], 'NUEVOS_ACUERDOS': Empty DataFrame
Columns: [NUEVOS_ACUERDOS]
Index: [], 'NUEVOS_SOCIOS': Empty DataFrame
Columns: [NUEVOS_SOCIOS]
Index: [], 'STATUS_SOCIOS': Empty DataFrame
Columns: [STATUS_SOCIO

In [5]:
qc.overwrite_acuerdos_prestamos(dict_session['PRÉSTAMO'], id_grupo, sesion_list)

     PRÉSTAMO AMPLIACIÓN NUM_SESIONES Prestamo_id
1704    [1.0]        [0]          [6]      [4034]
1834    [1.0]        [0]          [6]      [4044]
1844    [1.0]        [0]          [6]      [4054]
1854    [2.0]        [0]          [6]      [4064]
1864    [2.0]        [0]          [6]      [4074]
1894    [1.0]        [0]          [6]      [4084]
1914    [3.0]        [0]          [6]      [4094]
1924    [4.0]        [0]          [6]      [4104]
   Acuerdo_id  Grupo_id Fecha_acuerdos Fecha_acuerdos_fin  Status  \
0        1344      1744     2023-05-22         0000-00-00       0   
1        1404      1744     2023-05-22         0000-00-00       0   
2        1414      1744     2023-05-22         0000-00-00       0   
3        1424      1744     2023-05-22         0000-00-00       0   
4        1434      1744     2023-05-22         0000-00-00       0   
5        1444      1744     2023-05-22         0000-00-00       1   

   Periodo_reuniones  Periodo_cargos  Limite_inasistencias  Minimo

In [15]:
monitor_grupo = CDM.CAF_DB_Monitor(id_grupo)

In [16]:
monitor_grupo.limite_credito_socios

,Tipo_socio,Status,Acciones,Socio_id,monto_total,num_prestamos,Limite_credito_final,Limite_credito_teorico,puede_pedir_prestamo
0,ADMIN,1,200,1324,0,0,800,800,True
1,SUPLENTE,1,200,1334,0,0,800,800,True
2,SOCIO,1,200,1344,0,0,800,800,True
3,SOCIO,1,200,1354,0,0,800,800,True
4,SOCIO,1,200,1364,0,0,800,800,True
5,SOCIO,1,200,1374,0,0,800,800,True
6,SOCIO,1,200,1384,0,0,800,800,True
7,SOCIO,1,200,1394,0,0,800,800,True


In [17]:
start_month = 1
end_month = 4

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    try:
        mt.test_sesion_months(xls_name, month, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    [status, dfs] = monitor_grupo.compara_db_monitor()
    if not status:
        print('Algo no coincidió')
        break

month:  1
month:  2
month:  3
     STATUS_SOCIOS
1384        [-1.0]
1384
-1
ganancias No coinciden monitor y db
Algo no coincidió


In [9]:
monitor_grupo.g

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Estatus_ampliacion,Num_sesiones,Sesiones_restantes,Estatus_prestamo,Socio_id,...,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono,Status_socio
0,3134,500.0,0,20.0,0,False,6,4,0,1384,...,1,0.0,1,1.5,10.0,0,0,0,0,0


In [12]:
monitor_grupo.socios_acciones.loc[monitor_grupo.socios_acciones['Socio_id'] == 1324, 'Status'].values[0]

1

In [18]:
comp = dfs[0].compare(dfs[1])
comp

Monto_ganancia          
             self     other
24           1.25  1.428571
25           1.25  1.428571
26           1.25  1.428571
27           1.25  1.428571
28           1.25  1.428571
29           1.25  1.428571
30           1.25  1.428571

In [19]:
monitor_grupo.ganancias

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
0,49264,0.000000,0,1324,6994
1,49274,0.000000,0,1334,6994
2,49284,0.000000,0,1344,6994
3,49294,0.000000,0,1354,6994
4,49304,0.000000,0,1364,6994
5,49314,0.000000,0,1374,6994
6,49324,0.000000,0,1384,6994
7,49334,0.000000,0,1394,6994
8,52324,0.000000,0,1324,7434
9,52334,0.000000,0,1334,7434


In [ ]:
monitor_grupo

In [18]:
monitor_grupo.multas

,Multa_id,Monto_multa,Descripcion,Status,Sesion_id,Socio_id,Pago_en_sesion
0,1524,10.0,Multa # 1 socio 1534 grupo: 1574,1,7114,1534,0
1,1534,10.0,Multa # 1 socio 1584 grupo: 1574,1,7114,1584,0
2,1544,20.0,Multa # 1 socio 1614 grupo: 1574,1,7134,1614,0


In [22]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Estatus_ampliacion,Num_sesiones,Sesiones_restantes,Estatus_prestamo,Socio_id,...,Interes_morosidad,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono
0,2894,1000.0,1000.0,60.0,60.0,False,6,3,1,1574,...,0.0,0,None,1,1.5,0.0,0,0,0,0
1,2904,600.0,400.0,36.0,24.0,False,6,3,0,1524,...,0.0,0,None,1,1.5,12.0,12.0,12.0,200.0,212.0
2,2914,590.0,400.4,35.4,23.6,False,6,3,0,1554,...,0.0,0,None,1,1.5,11.8,11.8,11.8,200.2,212.0
3,2924,590.0,400.4,35.4,23.6,False,6,3,0,1604,...,0.0,0,None,1,1.5,11.8,11.8,11.8,200.2,212.0
4,2934,675.0,225.5,27.0,13.5,False,6,4,0,1544,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
5,2944,675.0,225.5,27.0,13.5,False,6,4,0,1564,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
6,2954,675.0,225.5,27.0,13.5,False,6,4,0,1584,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
7,2964,675.0,225.5,27.0,13.5,False,6,4,0,1594,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
8,2974,1000.0,0,20.0,0,False,6,5,0,1534,...,0.0,0,None,1,1.5,20.0,0,0,0,0
9,2984,1200.0,0,24.0,0,False,6,5,0,1574,...,0.0,0,None,1,1.5,24.0,0,0,0,0


In [19]:
monitor_grupo.ganancias

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
0,49504,0.000000,0,1524,7034
1,49514,0.000000,0,1534,7034
2,49524,0.000000,0,1544,7034
3,49534,0.000000,0,1554,7034
4,49544,0.000000,0,1564,7034
5,49554,0.000000,0,1574,7034
6,49564,0.000000,0,1584,7034
7,49574,0.000000,0,1594,7034
8,49584,0.000000,0,1604,7034
9,49594,0.000000,0,1614,7034


In [13]:
monitor_grupo.sesiones

[{'Sesion_id': 2174},
 {'Sesion_id': 6224},
 {'Sesion_id': 6234},
 {'Sesion_id': 6244},
 {'Sesion_id': 6254},
 {'Sesion_id': 6264},
 {'Sesion_id': 6274},
 {'Sesion_id': 6284},
 {'Sesion_id': 6294},
 {'Sesion_id': 6304}]

In [21]:
monitor_grupo.ganancias.groupby('Sesion_id').sum()

,Ganancias_id,Monto_ganancia,Entregada,Socio_id
Sesion_id,,,,
6994,394392,0.0,0,10872
7434,418872,0.0,0,10872
7444,419512,10.0,0,10872
7454,367598,10.0,0,9488
